In [2]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'b54bcf4d-1bca-4e8e-9a24-22ff2c3d462c',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

{'status': {'timestamp': '2025-02-01T14:01:58.185Z', 'error_code': 0, 'error_message': None, 'elapsed': 1, 'credit_count': 1, 'notice': None}, 'data': [{'id': 633, 'name': 'senlc5af6p', 'symbol': '87maouv2fln', 'slug': 'p4hqdhkq0cn', 'cmc_rank': 2968, 'num_market_pairs': 8520, 'circulating_supply': 1976, 'total_supply': 4151, 'max_supply': 5500, 'infinite_supply': None, 'last_updated': '2025-02-01T14:01:58.185Z', 'date_added': '2025-02-01T14:01:58.185Z', 'tags': ['o9n0a41skm', 'kmhn23tr6po', 'dpdnv09cn4t', 'saaj42ohv9b', 'avrug1d6mlr', 'meazmc1crvs', 'bx698o79riu', 'v8hjpmvmd59', '2vzzjjmzljg', 'ljh9lteeia'], 'platform': None, 'self_reported_circulating_supply': None, 'self_reported_market_cap': None, 'quote': {'USD': {'price': 0.057168067912672926, 'volume_24h': 4338, 'volume_change_24h': 0.14462653020408345, 'percent_change_1h': 0.8869418692815936, 'percent_change_24h': 0.46049805913197517, 'percent_change_7d': 0.5862936978132847, 'market_cap': 0.3974123626408903, 'market_cap_dominan